In [28]:
import sqlite3
import pandas as pd 
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

# Load environment variables
load_dotenv(find_dotenv())


True

In [4]:
client = OpenAI()


In [5]:
conn = sqlite3.connect('rca_data.db')
cursor = conn.cursor()

In [6]:
df = pd.read_csv('RCA_synthetic_data_expanded.csv')

df.to_sql('rca_data', conn, if_exists='replace', index=False)

110

In [7]:
# Check what schema was created
cursor = conn.cursor()
cursor.execute("SELECT sql FROM sqlite_master WHERE name='rca_data'")
print(cursor.fetchone()[0])

CREATE TABLE "rca_data" (
"Asset" TEXT,
  "Area" TEXT,
  "Equipment" TEXT,
  "RCA ID" TEXT,
  "Failure Event Details" TEXT,
  "Impact (AUD)" INTEGER,
  "Downtime (hours)" INTEGER,
  "Root Cause" TEXT,
  "Action" TEXT,
  "Action Status" TEXT
)


In [8]:
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(rca_data)")
rows = cursor.fetchall()
print("table name: rca_data\n" + "\n".join([f"{r[1]} ({r[2]})" for r in rows]))

table name: rca_data
Asset (TEXT)
Area (TEXT)
Equipment (TEXT)
RCA ID (TEXT)
Failure Event Details (TEXT)
Impact (AUD) (INTEGER)
Downtime (hours) (INTEGER)
Root Cause (TEXT)
Action (TEXT)
Action Status (TEXT)


In [9]:
# Create a metadata table for documentation
cursor.execute("""
    CREATE TABLE IF NOT EXISTS column_metadata (
        table_name TEXT NOT NULL,
        column_name TEXT NOT NULL,
        description TEXT,
        data_type TEXT,
        constraints TEXT,
        example_value TEXT,
        notes TEXT,
        PRIMARY KEY (table_name, column_name)
    )
""")
               
metadata = [    
    ('rca_data', 'Asset', 
     'Mine or facility name where the failure occurred', 
     'TEXT', 
     'NOT NULL', 
     'Mine A, Mine B', 
     'Identifies the specific mine/site asset; used for filtering investigations by location'),
    
    ('rca_data', 'Area', 
     'Operational area or department within the asset', 
     'TEXT', 
     'NOT NULL', 
     'Processing Plant, Rail Loading, Underground Operations, Tailings Dam, Haul Road', 
     'Categorizes failures by operational zone; helps identify high-risk areas'),
    
    ('rca_data', 'Equipment', 
     'Specific equipment name and identifier that experienced failure', 
     'TEXT', 
     'NOT NULL', 
     'Crusher 1, Pump Station 2, Conveyor Belt 8, Dump Truck 15', 
     'Equipment type and number for precise identification; includes both equipment category and unit number'),
    
    ('rca_data', 'RCA ID', 
     'Root Cause Analysis investigation unique identifier', 
     'TEXT', 
     'NOT NULL', 
     'RCA 1, RCA 2, RCA 10, RCA 50', 
     'Groups all root causes for a single failure event; multiple rows may share the same RCA ID if the investigation identified multiple contributing root causes'),
    
    ('rca_data', 'Failure Event Details', 
     'Comprehensive narrative description of the failure incident', 
     'TEXT', 
     'NOT NULL', 
     'At 1831H on 30th October, 2025, Equipment 1 started to trip on high torque...', 
     'Includes timestamp (date and time), symptoms, sequence of events, investigation findings, and any secondary damage; provides complete context for understanding the failure'),
    
    ('rca_data', 'Impact (AUD)', 
     'Total financial impact in Australian Dollars', 
     'INTEGER', 
     'None', 
     '1000000, 2500000, 850000, 420000', 
     'Aggregate cost including production losses, repair costs, labor, parts, and any secondary damages; used for prioritizing corrective actions and calculating ROI'),
    
    ('rca_data', 'Downtime (hours)', 
     'Total equipment downtime duration in hours', 
     'INTEGER', 
     'None', 
     '32, 96, 48, 18', 
     'Measures operational impact; from failure initiation to equipment return to service; used for availability and reliability metrics'),
    
    ('rca_data', 'Root Cause', 
     'Identified underlying cause of the failure', 
     'TEXT', 
     'NOT NULL', 
     'Misalignment, Overloading, Inadequate lubrication schedule, Operator error', 
     'Specific root cause identified through investigation; one failure event may have multiple root causes, each stored as a separate row with the same RCA ID'),
    
    ('rca_data', 'Action', 
     'Recommended corrective or preventive action', 
     'TEXT', 
     'NOT NULL', 
     'Implement routine maintenance, Install monitoring system, Enhance operator training', 
     'Specific action designed to address the identified root cause and prevent recurrence; each root cause has its own corresponding action'),
    
    ('rca_data', 'Action Status', 
     'Current implementation status of the corrective action', 
     'TEXT', 
     'DEFAULT In Progress', 
     'Completed, In Progress', 
     'Tracks whether the corrective action has been fully implemented; defaults to "In Progress" for new entries')
]

cursor.executemany("""
    INSERT OR REPLACE INTO column_metadata 
    (table_name, column_name, description, data_type, constraints, example_value, notes)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", metadata)

conn.commit()

In [10]:

rows = conn.execute("SELECT * FROM column_metadata").fetchall()
meta_schema = "table name: rca_data\n" + "\n".join([f"{r[1]}, Description: {r[2]}, Type: {r[3]}" for r in rows])

In [11]:
# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:")
for table in tables:
    print(f"  - {table[0]}")

Tables in database:
  - column_metadata
  - rca_data


In [12]:
cursor.execute("SELECT * FROM rca_data")
all_users = cursor.fetchall()
all_users

[('Mine A',
  'Processing Plant',
  'Crusher 1',
  'RCA 1',
  'At 1831H on the 30th October, 2025,Equipment 1 started to trip on high torque. It continued to experienced multiple tripping until 2100H on the 2nd November. Among them, longest downtime was 32.12 hours.',
  1000000,
  32,
  'Misalignment',
  'Implement routine maintenance',
  'Completed'),
 ('Mine A',
  'Processing Plant',
  'Crusher 1',
  'RCA 1',
  'At 1831H on the 30th October, 2025,Equipment 1 started to trip on high torque. It continued to experienced multiple tripping until 2100H on the 2nd November. Among them, longest downtime was 32.12 hours.',
  1000000,
  32,
  'Overloading',
  'Implement interlock to prevent overloading',
  'Completed'),
 ('Mine A',
  'Processing Plant',
  'Crusher 1',
  'RCA 2',
  'At 0645H on 15th March, 2025, Crusher 1 experienced catastrophic bearing failure during routine operation. Metal fragments were detected in the oil sample analysis conducted on 10th March. The bearing completely sei

In [ ]:
query = "What are the crusher failure in 2025?"
prompt = f"""
You are a SQL assistant. Given the schema and the user's question, write a SQL query for SQLite.

Schema:
{meta_schema}

User question:
{query}

Respond with the SQL only.
"""
response = client.chat.completions.create(
    model="gpt-5",
    messages=[{"role": "user", "content": prompt}],
)

sql_gen_1 = response.choices[0].message.content.strip()

In [64]:
sql_gen_1

'SELECT\n  "RCA ID",\n  Asset,\n  Area,\n  Equipment,\n  "Failure Event Details",\n  "Impact (AUD)",\n  "Downtime (hours)",\n  "Root Cause",\n  "Action",\n  "Action Status"\nFROM rca_data\nWHERE Equipment LIKE \'%Crusher%\'\n  AND (\n    "Failure Event Details" LIKE \'%2025%\'\n    OR "RCA ID" LIKE \'%2025%\'\n  );'

In [65]:
output = cursor.execute(sql_gen_1).fetchall()
output

[('RCA 1',
  'Mine A',
  'Processing Plant',
  'Crusher 1',
  'At 1831H on the 30th October, 2025,Equipment 1 started to trip on high torque. It continued to experienced multiple tripping until 2100H on the 2nd November. Among them, longest downtime was 32.12 hours.',
  1000000,
  32,
  'Misalignment',
  'Implement routine maintenance',
  'Completed'),
 ('RCA 1',
  'Mine A',
  'Processing Plant',
  'Crusher 1',
  'At 1831H on the 30th October, 2025,Equipment 1 started to trip on high torque. It continued to experienced multiple tripping until 2100H on the 2nd November. Among them, longest downtime was 32.12 hours.',
  1000000,
  32,
  'Overloading',
  'Implement interlock to prevent overloading',
  'Completed'),
 ('RCA 2',
  'Mine A',
  'Processing Plant',
  'Crusher 1',
  'At 0645H on 15th March, 2025, Crusher 1 experienced catastrophic bearing failure during routine operation. Metal fragments were detected in the oil sample analysis conducted on 10th March. The bearing completely sei

In [13]:
from typing import Tuple
def evaluate_and_refine_sql(
    question: str,
    sql_query: str,
    df: pd.DataFrame,
    schema: str,
    model: str = "gpt-5",
) -> Tuple[str, str]:
    """
    Evaluate whether the SQL result answers the user's question and,
    if necessary, propose a refined version of the query.
    Returns (feedback, refined_sql).
    """
    prompt = f"""
You are a SQL reviewer and refiner.

User asked:
{question}

Original SQL:
{sql_query}

SQL Output:
{df.to_markdown(index=False)}

Table Schema:
{schema}

Step 1: Briefly evaluate if the SQL output answers the user's question.
Step 2: If the SQL could be improved, provide a refined SQL query.
If the original SQL is already correct, return it unchanged.

Return a strict JSON object with two fields:
- "feedback": brief evaluation and suggestions
- "refined_sql": the final SQL to run
"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )

    import json
    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # Fallback if the model does not return valid JSON:
        # use the raw content as feedback and keep the original SQL
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql


In [71]:
feedback, refined_sql = evaluate_and_refine_sql(
        question=query,
        sql_query=sql_gen_1,
        df=df,
        schema=meta_schema,
    )

print("📝 Reflect on V1 SQL/output:\n" + feedback)
print("🔁 Write V2 query:\n" + refined_sql)

📝 Reflect on V1 SQL/output:
Partially answers the question. It correctly targets crusher equipment but relies on RCA ID to infer the year, which is unreliable and appears to pull in non-crusher/non-2025 rows. Filter only by Equipment containing 'Crusher' and by the event narrative mentioning 2025. If available, use a proper event date column instead of parsing text, and consider deduplicating RCAs if multiple root-cause rows exist.
🔁 Write V2 query:
SELECT
  "RCA ID",
  Asset,
  Area,
  Equipment,
  "Failure Event Details",
  "Impact (AUD)",
  "Downtime (hours)",
  "Root Cause",
  "Action",
  "Action Status"
FROM rca_data
WHERE Equipment LIKE '%Crusher%'
  AND "Failure Event Details" LIKE '%2025%';


In [74]:
q = refined_sql.strip().removeprefix("```sql").removesuffix("```").strip()
try:
    print(pd.read_sql_query(q, conn))
except Exception as e:
    print(pd.DataFrame({"error": [str(e)]}))


    RCA ID   Asset              Area        Equipment  \
0    RCA 1  Mine A  Processing Plant        Crusher 1   
1    RCA 1  Mine A  Processing Plant        Crusher 1   
2    RCA 2  Mine A  Processing Plant        Crusher 1   
3    RCA 8  Mine B  Processing Plant    Jaw Crusher 2   
4   RCA 18  Mine B  Crushing Station  Cone Crusher 17   
5   RCA 21  Mine B  Crushing Station   Jaw Crusher 17   
6   RCA 33  Mine B  Crushing Station    Jaw Crusher 7   
7   RCA 45  Mine B  Processing Plant       Crusher 18   
8   RCA 45  Mine B  Processing Plant       Crusher 18   
9   RCA 52  Mine A  Processing Plant        Crusher 5   
10  RCA 52  Mine A  Processing Plant        Crusher 5   

                                Failure Event Details  Impact (AUD)  \
0   At 1831H on the 30th October, 2025,Equipment 1...       1000000   
1   At 1831H on the 30th October, 2025,Equipment 1...       1000000   
2   At 0645H on 15th March, 2025, Crusher 1 experi...       2500000   
3   On 3rd September, 2025 at 1

In [26]:
def database_interpreter(query: str, sql_gen_ref: pd.DataFrame, metadata: str, model: str = "gpt-5"):

    prompt = f"""
    You are an expert data analyst reviewing SQL query results. You have access to a database schema and query results. Your task is to provide a clear, natural language answer to the user's question based on the data.

    DATABASE SCHEMA:
    {metadata}

    QUERY RESULTS:
    {sql_gen_ref}

    USER QUESTION:
    {query}

    INSTRUCTIONS:
    1. Analyze the query results in the context of the schema
    2. Provide a clear, concise answer in natural language
    3. Include relevant numbers, dates, and specific details from the results
    4. If the results are empty, explain that no data was found
    5. Format your response to be easy to read and understand
    6. Do not include SQL syntax in your answer - only natural language

    YOUR ANSWER:

    """
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt}],
    )

    output = response.choices[0].message.content.strip()
    return output

In [29]:
#Full workflow
def database_agent(query: str, model: str = "gpt-5"): 
    # query = "What are the pump failures in Mine B in 2025?"
    prompt = f"""
    You are a SQL assistant. Given the schema and the user's question, write a SQL query for SQLite.

    Schema:
    {meta_schema}

    User question:
    {query}

    Respond with the SQL only.
    """
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt}],
    )

    sql_gen_1 = response.choices[0].message.content.strip()

    feedback, refined_sql = evaluate_and_refine_sql(
            question=query,
            sql_query=sql_gen_1,
            df=df,
            schema=meta_schema,
            model=model
        )

    print("📝 Reflect on V1 SQL/output:\n" + feedback)
    print("🔁 Write V2 query:\n" + refined_sql)

    q = refined_sql.strip().removeprefix("```sql").removesuffix("```").strip()
    try:
        sql_gen_ref = pd.read_sql_query(q, conn)
    except Exception as e:
        print(pd.DataFrame({"error": [str(e)]}))

    output = database_interpreter(query, sql_gen_ref, metadata=meta_schema, model=model)
    return output


In [33]:
query = "What are the top 5 downtimes in Mine A in terms of impact?"
output = database_agent(query, model="gpt-5-mini")

display(Markdown(query))
display(Markdown(output))

📝 Reflect on V1 SQL/output:
The original query will return the top 5 rows for Mine A ordered by impact, which generally answers the question. However the result set may include duplicate RCA rows (same "RCA ID" repeated across multiple root-cause/action rows) and it only returns a small set of columns. To ensure you get the top 5 distinct failure events by impact, deduplicate by "RCA ID" (or use DISTINCT ON / a window function depending on your RDBMS) and return the related fields. The refined query below aggregates per RCA ID and picks the max impact and downtime per event.
🔁 Write V2 query:
SELECT
  "RCA ID",
  MAX(Asset)                           AS asset,
  MAX(Area)                            AS area,
  MAX(Equipment)                       AS equipment,
  MAX("Impact (AUD)")                AS impact_aud,
  MAX("Downtime (hours)")            AS downtime_hours,
  MAX("Failure Event Details")       AS failure_event_details,
  MAX("Root Cause")                  AS root_cause,
  MAX("A

What are the top 5 downtimes in Mine A in terms of impact?

Here are the top 5 Mine A downtime events by financial impact:

1) Impact: AUD 3,344,968 — Equipment: Grader 10 (Haul Road) — Downtime: 16 h — Date/time: 29 Sep 2025 05:26 — RCA ID: RCA 70 — Root cause: Software malfunction — Action status: In Progress
2) Impact: AUD 3,135,004 — Equipment: Clarifier 6 (Water Treatment) — Downtime: 72 h — Date/time: 22 Mar 2025 12:13 — RCA ID: RCA 54 — Root cause: Thermal stress — Action status: Completed
3) Impact: AUD 2,950,559 — Equipment: Loader 6 (Underground Operations) — Downtime: 118 h — Date/time: 12 May 2025 10:10 — RCA ID: RCA 10 — Root cause: Material blockage — Action status: Completed
4) Impact: AUD 2,500,000 — Equipment: Crusher 1 (Processing Plant) — Downtime: 96 h — Date/time: 15 Mar 2025 06:45 — RCA ID: RCA 2 — Root cause: Inadequate lubrication schedule — Action status: In Progress
5) Impact: AUD 2,239,551 — Equipment: Loader 13 (Underground Operations) — Downtime: 53 h — Date/time: 24 Sep 2025 10:14 — RCA ID: RCA 31 — Root cause: Operator error — Action status: In Progress

Totals for these top 5: AUD 14,170,082 impact and 355 hours of downtime.